In [41]:
import numpy as np
import csv
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [42]:
data_path = "recorded-data"
steering_offset = 0.25
model_name = "xt-summit-model.h5"

#### Utility methods

In [43]:
def load_lines(path):
    lines = []
    with open(path + "/driving_log.csv") as datafile:
        reader = csv.reader(datafile)
        for line in reader:
            lines.append(line)
    return lines

def extract_images(line):
    #print(data_path + "/IMG/" + line[0].replace("\\", "/").split("/")[-1])
    center_image = cv2.imread(data_path + "/IMG/" + line[0].replace("\\", "/").split("/")[-1])
    left_image = cv2.imread(data_path + "/IMG/" + line[1].replace("\\", "/").split("/")[-1])
    right_image = cv2.imread(data_path + "/IMG/" + line[2].replace("\\", "/").split("/")[-1])
    return (center_image, left_image, right_image)

def extract_steering_angles(line):
    steering_angle_center = float(line[3])
    steering_angle_left = steering_angle_center + steering_offset
    steering_angle_right = steering_angle_center - steering_offset
    return (steering_angle_center, steering_angle_left, steering_angle_right)

def get_data_without_generator(path, lines):
    images = []
    steering_angles = []
    for line in lines:
        center_image, left_image, right_image = extract_images(line)
        center_image, left_image,right_image = convert_to_yuv(center_image), convert_to_yuv(left_image), convert_to_yuv(right_image)
        center_image, left_image,right_image = resize(center_image, (0.5, 0.5)), resize(left_image,(0.5, 0.5)), resize(right_image, (0.5, 0.5))
        center_image, left_image,right_image = crop_image(center_image), crop_image(left_image), crop_image(right_image)
        steering_angle_center, steering_angle_left, steering_angle_right = extract_steering_angles(line)
        images.extend([center_image, left_image, right_image])
        steering_angles.extend([steering_angle_center, steering_angle_left, steering_angle_right])
    return np.array(images), np.array(steering_angles)

def convert_to_yuv(image):
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    return converted_image

def crop_image(image):
    mask = np.ones_like(image)
    xsize, ysize = image.shape[1], image.shape[0]
    vertices = ((0, 0.4375*ysize), (0, ysize), (xsize, ysize), (xsize, 0.4375*ysize))
    vertexArr = []
    for vertex in vertices:
        vertexArr.append((vertex[0], vertex[1]))
    vertexArr = np.array([vertexArr], dtype=np.int32)
    
    cv2.fillPoly(mask, vertexArr, (255, 255, 255))
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def resize(image, scale_factor, size=(200, 66)):
    #resized_image = cv2.resize(image, (0,0), fx=scale_factor[0], fy=scale_factor[1]) 
    resized_image = cv2.resize(image, size) 
    return resized_image

def flip_vertical(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image

In [44]:
def conv_layer(input, channel_in, channel_out, layer_name="convLayer", weight_name="W", bias_name="B", 
               conv_kernel_size=5, use_pool=True):
    with tf.name_scope(layer_name):
        w = tf.Variable(tf.zeros([conv_kernel_size, conv_kernel_size, channel_in, channel_out]), name=weight_name)
        b = tf.Variable(tf.zeros(channel_out), name=bias_name)
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="VALID")
        conv = tf.nn.bias_add(conv, b)
        conv = tf.nn.relu(conv)
        if (use_pool):
            pool = tf.nn.avg_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        else:
            pool = conv
        tf.summary.histogram("Weights", w)
        tf.summary.histogram("Biases", b)
        tf.summary.histogram("Activation", conv)
        return pool

In [45]:
def full_layer(input, channel_in, channel_out, layer_name="fullLayer", weight_name="W", bias_name="B"):
    with tf.name_scope(layer_name):
        w = tf.Variable(tf.zeros([channel_in, channel_out]), name=weight_name)
        b = tf.Variable(tf.zeros(channel_out), name=bias_name)
        full = tf.add(tf.matmul(input, w), b)
        full = tf.nn.relu(full)
        tf.summary.histogram("Weights", w)
        tf.summary.histogram("Biases", b)
        tf.summary.histogram("Activation", full)
        return full

In [52]:
batch_size = 128
num_of_epochs = 5
dropout_keep_prob = 0.5

In [47]:
from tensorflow.contrib.layers import flatten
def run_network_arch (x):
    
    conv1 = conv_layer(x, 3, 24, layer_name="convLayer1", weight_name="weightConv1", bias_name="biasConv1")
    
    conv2 = conv_layer(conv1, 24, 36, layer_name="convLayer2", weight_name="weightConv2", bias_name="biasConv2")
    
    conv3 = conv_layer(conv2, 36, 48, layer_name="convLayer3", weight_name="weightConv3", bias_name="biasConv3")
    
    conv4 = conv_layer(conv3, 48, 64, layer_name="convLayer4", weight_name="weightConv4", bias_name="biasConv4", 
                       conv_kernel_size=3)
    
    flat = flatten(conv4)
    
    fc1 = full_layer(flat, 576, 100, layer_name="fullLayer1", weight_name="weightFull1", bias_name="biasFull1")
    
    fc1 = tf.nn.dropout(fc1, keep_prob=dropout_keep_prob)
    
    fc2 = full_layer(fc1, 100, 50, layer_name="fullLayer2", weight_name="weightFull2", bias_name="biasFull2")
    
    fc2 = tf.nn.dropout(fc2, keep_prob=dropout_keep_prob)
      
    logits = full_layer(fc2, 50, 1, layer_name="fullLayer3", weight_name="weightFull3", bias_name="biasFull3")
    
    #tf.summary.image("Logit", fc2, 50)
    
    return logits

### Declare TensorFlow model

In [48]:
from tensorflow.contrib.layers import xavier_initializer
import tensorflow as tf
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 66, 200, 3), name="InputImages")
tf.summary.image("Input", x, 3)
y = tf.placeholder(tf.float32, (None), name="OutputSteeringAngle")

### Training Pipeline

In [49]:
logits = run_network_arch(x)
with tf.name_scope("Entropy"):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits, y)
    loss_operation = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("Loss", loss_operation)
with tf.name_scope("Optimizer"):
    optimizer = tf.train.AdamOptimizer()
    training_operation = optimizer.minimize(loss_operation)

### Evalute performance

In [50]:
with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy", accuracy_operation)

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset + batch_size], y_data[offset:offset + batch_size]
        batch_y = np.array(batch_y, dtype=np.float64).reshape(len(batch_y), 1)
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Main module

In [51]:
def main():
    lines = load_lines(data_path)[1:]
    training_set_lines, validation_set_lines = train_test_split(lines, test_size=0.2)
    
    nb_training = len(training_set_lines)*6
    nb_validation = len(validation_set_lines)*6

    training_images, steering_angles = get_data_without_generator(data_path, lines[0:500])
    return (training_images, steering_angles)
data_path = "data-from-udacity"
#main()



### Train in TensorFlow session

In [54]:
from sklearn.utils import shuffle
saver = tf.train.Saver()
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("tensorboard-graphs/tensorboard/2")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_of_epochs):
        images, angles = main()
        training_images, valid_images, steering_angles, valid_steering_angles = train_test_split(images, angles)
        training_images, steering_angles = shuffle(training_images, steering_angles)
        for offset in range(0, len(training_images), batch_size):
            current_input_batch = training_images[offset: offset + batch_size]
            current_labels_batch = steering_angles[offset: offset + batch_size]
            current_labels_batch = np.array(current_labels_batch, dtype=np.float64).reshape(len(current_labels_batch), 1)
            sess.run(training_operation, feed_dict={x: current_input_batch, y: current_labels_batch})
            s = sess.run(merged_summary, feed_dict={x: current_input_batch, y: current_labels_batch})
            writer.add_summary(s, i)
        training_accuracy = evaluate(training_images, steering_angles)
        valid_steering_angles = np.array(valid_steering_angles).reshape(len(valid_steering_angles), 1)
        training_loss = sess.run(loss_operation, feed_dict={x: valid_images, y: valid_steering_angles})
        
        print("Epoch ", i, "is complete!")
        print("Validation loss for Epoch " +  str(i) + " is: ", training_loss)
    saver.save(sess, './xt-summit.ckpt')
    writer.add_graph(sess.graph)
    

Epoch  0 is complete!
Validation loss for Epoch 0 is:  0.693147
Epoch  1 is complete!
Validation loss for Epoch 1 is:  0.693147
Epoch  2 is complete!
Validation loss for Epoch 2 is:  0.693147
Epoch  3 is complete!
Validation loss for Epoch 3 is:  0.693147
Epoch  4 is complete!
Validation loss for Epoch 4 is:  0.693147
